In [1]:
#Importando la libreria para web
from bs4 import BeautifulSoup
import requests
#Importando libreria manejo de datos
import pandas as pd
import numpy as np
#Importando libreria graficar mapas
import folium
#Definimos la url de wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#Leemos el html como texto
data = requests.get(url).text
#Leemos el html con BS
soup = BeautifulSoup(data,"html.parser")
#Importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans
#Modulos matplotlib grafica final
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#print(soup.prettify())
#Leyendo la tabla
table = soup.find("table")
dtBarToronto = pd.DataFrame(columns=['postalCode', 'borough', 'neighborhood'])
#Ciclo para las filas
for row in table.find_all("tr"):
    #Ciclo para las columnas
    for col in row.find_all("td"):
        #Condicional ignorar celdas no asignadas
        if col.find(string="Not assigned") == None:
            #Obteniendo lista de strings en la celda
            colStrings = col.p.strings
            neighborhood = ""
            #Ciclo para recorrer la lista de strings
            for i,colString in enumerate(colStrings):
                uniString = str(colString)
                #Primera posicion es el codigo postal
                if i == 0:
                    postalCode = uniString
                #Segunda posicion es el municipio
                elif i == 1:
                    borough = uniString
                else:
                    neighborhood = neighborhood + uniString

            #row_df = pd.DataFrame([postalCode,borough,])
            #df = pd.concat([row_df, df], ignore_index=True)
            neighborhood = neighborhood.strip().replace("(","").replace(")","").replace(" /",",")
            nuevaFila = {"postalCode":postalCode,"borough":borough,"neighborhood":neighborhood}
            dtBarToronto = dtBarToronto.append(nuevaFila,ignore_index=True)
            #print(postalCode,borough,neighborhood)
        else:
            print("Casilla No Asignada")


Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada


In [3]:
#Se asigna el valor del municipio a los barrios que se encuentran vacios
dtBarToronto['neighborhood'] = dtBarToronto['neighborhood'].fillna(dtBarToronto['borough'])

In [4]:
#Agrupando barrios por codigo postal y municipio
dtBarToronto = dtBarToronto.groupby(['postalCode','borough'])['neighborhood'].apply(','.join).reset_index()

In [5]:
#Verificamos que ninguno de los codigos postales aparece mas de una vez
dtBarToronto['postalCode'].value_counts(sort=True)
#La frecuencia de todos los codigos es 1

M1X    1
M2R    1
M3M    1
M6B    1
M6K    1
      ..
M6N    1
M1E    1
M5J    1
M5A    1
M5L    1
Name: postalCode, Length: 103, dtype: int64

In [6]:
#Tamaño del dataframe resultante
dtBarToronto.shape

(103, 3)

In [7]:
"""
import geocoder # import geocoder
print("prueba print")
for index,row in dtBarToronto.iterrows():
  # initialize your variable to None
  lat_lng_coords = None
  postal_code = row["postalCode"]
  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

  row["latitude"] = lat_lng_coords[0]
  row["longitude"] = lat_lng_coords[1]
  print(lat_lng_coords[0],lat_lng_coords[1])
"""

'\nimport geocoder # import geocoder\nprint("prueba print")\nfor index,row in dtBarToronto.iterrows():\n  # initialize your variable to None\n  lat_lng_coords = None\n  postal_code = row["postalCode"]\n  # loop until you get the coordinates\n  while(lat_lng_coords is None):\n    g = geocoder.google(\'{}, Toronto, Ontario\'.format(postal_code))\n    lat_lng_coords = g.latlng\n\n  row["latitude"] = lat_lng_coords[0]\n  row["longitude"] = lat_lng_coords[1]\n  print(lat_lng_coords[0],lat_lng_coords[1])\n'

In [8]:
#Leyendo el archivo excel con las coordenadas geoespaciales
df_spacial = pd.read_csv("Geospatial_Coordinates.csv")
df_spacial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
#Haciendo Join con el codigo postal
dtToronto = dtBarToronto.set_index("postalCode").join(df_spacial.set_index("Postal Code"))

In [10]:
#Volviendo el codigo postal una columna
dtToronto.reset_index(inplace=True)
dtToronto.head()

,postalCode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
#Generando mapa Toronto
map_toronto = folium.Map(location = [43.6532,-79.3832], zoom_start = 11)
#Añadiendo marcadores para cada barrio
for lat,lon,nei,bor in zip(dtToronto["Latitude"],dtToronto["Longitude"],dtToronto["neighborhood"],dtToronto["borough"]):
    label = '{},{}'.format(nei,bor)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(location=[lat,lon],radius=5,popup=label,color="green",fill=True,fill_color="#99ff99",parse_html=True).add_to(map_toronto)
#Graficando mapa
map_toronto

In [12]:
#Filtrando unicamente los municipios con la palabra "Toronto en el nombre"
toronto_data = dtToronto[dtToronto["borough"].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,postalCode,borough,neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [13]:
#Configuracion Foursquare
CLIENT_ID = 'VL21BT4LPJFFG0UHO5TJWRW0KXOUI5ZNOOC2T24OI544KJTU' # su ID de Foursquare
CLIENT_SECRET = 'EDWOK1HJXYY2TCSL5DNA1OQ0BG1PVBV01ZWJCDGFUFBSAOUC' # Secreto de Foursquare
VERSION = '20180605' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare


In [14]:
#Definimos funcion para obtener categoria de los lugares
def obtener_categoria(row):
    try:
        lista_categoria = row["categories"]
    except:
        lista_categoria = row["venue.categories"]
    if len(lista_categoria) == 0:
        return None
    else:
        return lista_categoria[0]["name"]


In [15]:
toronto_data.head()

,postalCode,borough,neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
#Definimos funcion para obtener los lugares para cada barrio
def obtener_lugares(names,latitudes,longitudes,radius=500):
    lista_lugares = []
    for name,lat,lon in zip(names,latitudes,longitudes):
        #crear la URL de solicitud de API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lon,radius,LIMIT)
        #solicitud GET
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #regresa solo información relevante de cada sitio cercano
        lista_lugares.append([(name,lat,lon,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in results])
        lugares_cercanos = pd.DataFrame([item for lugar in lista_lugares for item in lugar])
        lugares_cercanos.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(lugares_cercanos)


In [17]:
#Llamamos a la funcion anterior
lugares_toronto = obtener_lugares(toronto_data["neighborhood"],toronto_data["Latitude"],toronto_data["Longitude"])
lugares_toronto.shape

ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [ ]:
print("Hay {} categorias unicas en Toronto".format(len(lugares_toronto["Venue Category"].unique())))

Hay 233 categorias unicas en Toronto


In [ ]:
# generando una columna para cada categoria
toronto_onehot = pd.get_dummies(lugares_toronto[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Neighborhood'] = lugares_toronto['Neighborhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot= toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#obteniendo la frecuencia de cada categoria para cada barrio
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Business reply mail Processing Centre969 Easte...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.014925,0.0,0.0,0.0,0.0


In [ ]:
#Funcion para retornar lugares mas comunes
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Bakery,Coffee Shop,Farmers Market,Seafood Restaurant,Pharmacy,Cheese Shop,Restaurant,Beer Bar,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Nightclub,Climbing Gym,Restaurant,Burrito Place
2,Business reply mail Processing Centre969 Easte...,Gym / Fitness Center,Garden,Park,Light Rail Station,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Coffee Shop,Airport,Airport Food Court,Airport Gate,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Spa,Salad Place,Burger Joint,Bubble Tea Shop


In [ ]:
# establecer el número de agrupaciones
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# juntar manhattan_grouped con manhattan_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

toronto_merged.head() # revisar las ultimas columnas

,postalCode,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Wine Shop,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Café,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Sushi Restaurant,Light Rail Station,Liquor Store,Sandwich Place,Burrito Place,Fast Food Restaurant,Italian Restaurant,Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Gym / Fitness Center,Diner,Park,Middle Eastern Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Wine Shop,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [ ]:
# crear mapa
map_clusters = folium.Map(location=[43.6532,-79.3832], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters